### Figure 5: Power Spectra at each wave number

In [ ]:
import numpy as np
# from esda.moran import Moran,Moran_Local
# from libpysal.weights import lat2W
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
# import properscoring as ps
import warnings
from scipy import stats
import scipy.stats as stats
from scipy import interpolate
# from utils.evaluation import plot_histogram,calc_peak,calc_mean
# https://bertvandenbroucke.netlify.app/2019/05/24/computing-a-power-spectrum-in-python/
from sklearn.metrics import mean_squared_error
from math import sqrt
from utils.data import load_tc_data
# import xesmf as xe
warnings.filterwarnings("ignore")
# sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))

# TODO: plot all ensemble members individually

In [ ]:
def power_spectrum_dB(img):
	fx = np.fft.fft2(img)
	fx = fx[:img.shape[0]//2, :img.shape[1]//2] # disgard half of it because you can derive one half from the other?
	px = abs(fx)**2 # get the size of the amplitudes
	return 10 * np.log10(px) # is this the rainfall normalisation step?

def power_spectrum_dB_2(img):
	fx = np.fft.fft2(img)
	# fx = fx[:img.shape[0]//2, :img.shape[1]//2] # disgard half of it because you can derive one half from the other?
	px = abs(fx)**2 # get the size of the amplitudes
	return px

def log_spectral_batch(batch1):
	lsd_batch = []
	for i in range(batch1.shape[0]):
		lsd = power_spectrum_dB_2(
				batch1[i, :, :])
		lsd_batch.append(lsd)
	return np.array(lsd_batch)

In [ ]:
model = ''
# set mode
mode = 'validation'

# load datasets
real,inputs,pred,meta = load_tc_data(set='validation',results='ke_tracks')
real_2,inputs_2,pred_2,meta_2,inputs_og,pred_og,meta_og = load_tc_data(set='validation',results='kh_tracks')
modular_pred_2 = np.load('/user/home/al18709/work/gan_predictions_20/modular_part2_pred-opt_modular_test.npy')
meta_og = pd.read_csv('/user/work/al18709/tc_data_mswep_40/valid_meta.csv')
meta_valid = pd.read_csv('/user/work/al18709/tc_data_flipped/valid_meta.csv')
print(pred.shape)
print(inputs.shape)
print(real.shape)


In [ ]:
lsb_real = log_spectral_batch(real[:,:,:,0])
lsb_gan = log_spectral_batch(pred[:,:,:,0])
lsb_modular = log_spectral_batch(modular_pred_2[:,:,:,0])
# lsb_gan2 = log_spectral_batch(pred_2[:,:,:,0])
lsb_ganog = log_spectral_batch(pred_og[:,:,:,0])
print(lsb_real[0])


In [ ]:
def plot_power_spectrum(spectral_density,plot=True):
    """
    spectral density should probably be a 100x100 image
    """
	# https://bertvandenbroucke.netlify.app/2019/05/24/computing-a-power-spectrum-in-python/
    npix=100

    # return a one dimensional array containing the wave vectors for the numpy.fft.fftn call, in the correct order.
    kfreq = np.fft.fftfreq(npix) * npix
    
    # convert this to a two dimensional array matching the layout of the two dimensional Fourier image, we can use numpy.meshgrid
    kfreq2D = np.meshgrid(kfreq, kfreq)
    
    # Finally, we are not really interested in the actual wave vectors, but rather in their norm
    knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)

    # we no longer need the wave vector norms or Fourier image to be laid out as a two dimensional array, so we will flatten them
    knrm = knrm.flatten()
    fourier_amplitudes = spectral_density
    fourier_amplitudes = fourier_amplitudes.flatten()

    # bin the amplitudes in k space, we need to set up wave number bins
    # Note that the maximum wave number will equal half the pixel size of the image. 
    # This is because half of the Fourier frequencies can be mapped back to negative wave
    #  numbers that have the same norm as their positive counterpart.
    kbins = np.arange(0.5, npix//2+1, 1.)

    # The kbin array will contain the start and end points of all bins; the corresponding k values are the midpoints of these bins
    kvals = 0.5 * (kbins[1:] + kbins[:-1])

    # To compute the average Fourier amplitude (squared) in each bin, we can use scipy.stats
    Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                        statistic = "mean",
                                        bins = kbins)

    # Remember that we want the total variance within each bin. Right now, we only have the average power. To get the total power, we need to multiply with the volume in each bin (in 2D, this volume is actually a surface area)
    Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

    if plot == True:
        plt.loglog(kvals, Abins)
        plt.show
    else:
        return kvals,Abins
    

In [ ]:

def calc_power(lsb_pred):
	kvals_list = []
	Abins_list = []

	for lsb in lsb_pred:
		kvals,Abins = plot_power_spectrum(lsb,plot=False)
		kvals_list.append(kvals)
		Abins_list.append(Abins)

	kvals = np.array(kvals_list)
	Abins = np.array(Abins_list)
	return kvals,Abins


In [ ]:
kvals_real,Abins_real = calc_power(lsb_real)
kvals_gan,Abins_gan = calc_power(lsb_gan)
kvals_modular,Abins_modular = calc_power(lsb_modular)
# kvals_gan2,Abins_gan2 = calc_power(lsb_gan2)
kvals_ganog,Abins_ganog = calc_power(lsb_ganog)

In [ ]:

def mean_power(kvals,Abins):

	xnew = np.sort(kvals.flatten())
	Y = np.zeros((Abins.shape[0],len(xnew)))
	for i,y in enumerate(Abins):
		x = kvals[i]	
		f = interpolate.interp1d(x, y,bounds_error=False)
		ynew = f(xnew)
		print(ynew.shape)
		print('xnew shape',xnew.shape)
		Y[i] = ynew
	print(xnew.shape)
	print(Y.shape)
	# split up to process the mean
	Y_mean_1 = np.nanmean(Y[0:5000],axis=0)
	Y_mean_2 = np.nanmean(Y[5000:10000],axis=0)
	Y_mean_3 = np.nanmean(Y[10000:],axis=0)
	Y_mean = np.nanmean([Y_mean_1,Y_mean_2,Y_mean_3],axis=0)
	print(Y_mean.shape)
	# kvals = x, Abins = Y_mean
	return(xnew,Y_mean)


In [ ]:
# plt.plot(xnew,Y_mean)
# k,p_k = mean_power(kvals,Abins)
# TODO: this takes way too long
print(kvals_real[0])
print(kvals_real[1])
k_real,p_k_real = kvals_real[0],np.mean(Abins_real,axis=0)
k_gan,p_k_gan = kvals_gan[0],np.mean(Abins_gan,axis=0)
k_modular,p_k_modular = kvals_modular[0],np.mean(Abins_modular,axis=0)
K_gan2,p_k_gan2 = kvals_gan2[0],np.mean(Abins_gan2,axis=0)
K_ganog,p_k_ganog = kvals_ganog[0],np.mean(Abins_ganog,axis=0)


In [ ]:
k_gan_ensemble = np.zeros((20,50))
p_k_gan_ensemble = np.zeros((20,50))
for i in range(1):
	lsb_gan_i = log_spectral_batch(pred[:,:,:,i])
	kvals_gan_i,Abins_gan_i = calc_power(lsb_gan_i)
	k_gan,p_k_gan = kvals_gan[0],np.mean(Abins_gan,axis=0)
	k_gan_i,p_k_gan_i = kvals_gan[0],np.mean(Abins_gan_i,axis=0)
	k_gan_ensemble[i,:] = k_gan_i
	p_k_gan_ensemble[i,:] = p_k_gan_i


In [ ]:
k_gan_min,p_k_gan_min = k_gan_ensemble[0],np.min(p_k_gan_ensemble,axis=0)
k_gan_max,p_k_gan_max = k_gan_ensemble[0],np.max(p_k_gan_ensemble,axis=0)

k_gan_ensemble,p_k_gan_ensemble = k_gan_ensemble[0],np.mean(p_k_gan_ensemble,axis=0)


In [ ]:
# Most power is located at large scales - small wave numbers. Does this make sense? This would mean most of the 
# image is dominated by large structures i.e. the core?
# Towards smaller scales (larger wave numbers) the power drops off suddenly and in a linear (but it is a log scale? I guess it is a 
# double log) fashion. This is an expression of the fractal nature of the TC: at lower resolutions the same types of patterns return, 
# but at increasingly lower signal. There is a huge spike in the power spectrum at a wave number of ~300, or a size scale of about 3 
# pixels. This is the scale at which the smallest rainfall features at the edges of the large features manifest.
fig, axes = plt.subplots(1, 1, figsize=(10, 10), sharey=False)
plt.style.use('seaborn-ticks')
plt.rcParams["figure.figsize"] = (10,10)
sns.set_context("notebook")
# print(k)
# print(p_k)
def change_units(x):
	# npix * 1/k gives the units of pixels on the x axis
	# then we multiply by 10 because each pixel is 10km, the whole image is 1000km across
	# return 1000/x
	return x/1000

plt.loglog(change_units(k_gan_ensemble),p_k_gan_ensemble,color='#219ebc')
# plt.loglog(change_units(K_gan2),p_k_gan2,color='Green')
plt.loglog(change_units(K_ganog),p_k_ganog,color='Pink')

# plt.loglog(k_gan_ensemble[0],p_k_gan_ensemble[0],color='#219ebc')
# plt.loglog(k_gan_ensemble[1],p_k_gan_ensemble[1],color='#219ebc')
# plt.loglog(k_gan_ensemble[2],p_k_gan_ensemble[2],color='#219ebc')
# plt.loglog(k_gan_ensemble[3],p_k_gan_ensemble[3],color='#219ebc')
# plt.loglog(k_gan_ensemble[4],p_k_gan_ensemble[4],color='#219ebc')
axes.text(-0.1, 1.05, 'a.', transform=axes.transAxes, size=28, weight='bold')
plt.loglog(change_units(k_real),p_k_real,color='black')
plt.xlabel("$k$ ($km^{-1}$)",fontsize=30)
plt.xlim([0,0.1])
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.ylabel("$P(k)$",fontsize=30)
plt.legend(['WGAN_1D','WGAN_OG','HR obs'],frameon=True,fontsize=18)
# plt.fill_between(k_vaegan_ensemble, p_k_vaegan_min,p_k_vaegan_max,color='#57cc99',alpha=0.2)
# plt.fill_between(k_gan_ensemble, p_k_gan_min,p_k_gan_max,color='#219ebc',alpha=0.2)
plt.savefig('figure5-power_spectra-ke_tracks_24.png',bbox_inches='tight',dpi=600)
# plt.show

In [ ]:
# The unet and the hr_input images are blurrier than their real contourparts. We can see this in the power spectra graph as 
# the power spectra drops off when the wave numnber is around 2. The images lose details as we sample smaller pixel areas.

In [ ]:
print(p_k_gan_max)
# print(p_k_gan_min)
print(p_k_gan_ensemble)
# print(p_k_gan_std)

In [ ]:
# p_k_vaegan_std

In [ ]:
inputs_x_max = np.max(inputs_x,axis=(1,2))
max_idx = list(np.argpartition(inputs_x_max, -100)[-100:])
pred_vaegan_x = pred_vaegan_x[max_idx]
pred_gan_x = pred_gan_x[max_idx]
real_x = real_x[max_idx]
pred_vaegan_ensemble_x = pred_vaegan_ensemble_x[max_idx]
pred_gan_ensemble_x = pred_gan_ensemble_x[max_idx]
inputs_hr_x = inputs_x_hr[max_idx]
pred_cnn_x = pred_cnn_x[max_idx]

In [ ]:
lsb_real = log_spectral_batch(real_x)
lsb_gan = log_spectral_batch(pred_gan_x)
lsb_vaegan = log_spectral_batch(pred_vaegan_x)
lsb_unet = log_spectral_batch(pred_cnn_x)
lsb_inputs = log_spectral_batch(inputs_x_hr)

kvals_real,Abins_real = calc_power(lsb_real)
kvals_gan,Abins_gan = calc_power(lsb_gan)
kvals_vaegan,Abins_vaegan = calc_power(lsb_vaegan)
kvals_unet,Abins_unet = calc_power(lsb_unet)
kvals_inputs,Abins_inputs = calc_power(lsb_inputs)

k_real,p_k_real = kvals_real[0],np.mean(Abins_real,axis=0)
k_gan,p_k_gan = kvals_gan[0],np.mean(Abins_gan,axis=0)
k_vaegan,p_k_vaegan = kvals_vaegan[0],np.mean(Abins_vaegan,axis=0)
k_unet,p_k_unet = kvals_unet[0],np.mean(Abins_unet,axis=0)
k_inputs,p_k_inputs = kvals_inputs[0],np.mean(Abins_inputs,axis=0)

In [ ]:
k_gan_ensemble = np.zeros((20,50))
k_vaegan_ensemble = np.zeros((20,50))
p_k_gan_ensemble = np.zeros((20,50))
p_k_vaegan_ensemble = np.zeros((20,50))
for i in range(20):
	lsb_vaegan_i = log_spectral_batch(pred_vaegan_ensemble_x[:,:,:,i])
	lsb_gan_i = log_spectral_batch(pred_gan_ensemble_x[:,:,:,i])
	kvals_vaegan_i,Abins_vaegan_i = calc_power(lsb_vaegan_i)
	kvals_gan_i,Abins_gan_i = calc_power(lsb_gan_i)
	k_gan,p_k_gan = kvals_gan[0],np.mean(Abins_gan,axis=0)
	k_vaegan_i,p_k_vaegan_i = kvals_vaegan[0],np.mean(Abins_vaegan_i,axis=0)
	k_gan_i,p_k_gan_i = kvals_gan[0],np.mean(Abins_gan_i,axis=0)
	k_gan_ensemble[i,:] = k_gan_i
	p_k_gan_ensemble[i,:] = p_k_gan_i
	k_vaegan_ensemble[i,:] = k_vaegan_i
	p_k_vaegan_ensemble[i,:] = p_k_vaegan_i

In [ ]:
k_vaegan_min,p_k_vaegan_min = k_vaegan_ensemble[0],np.min(p_k_vaegan_ensemble,axis=0)
k_vaegan_max,p_k_vaegan_max = k_vaegan_ensemble[0],np.max(p_k_vaegan_ensemble,axis=0)
k_gan_min,p_k_gan_min = k_gan_ensemble[0],np.min(p_k_gan_ensemble,axis=0)
k_gan_max,p_k_gan_max = k_gan_ensemble[0],np.max(p_k_gan_ensemble,axis=0)

k_vaegan_ensemble,p_k_vaegan_ensemble = k_vaegan_ensemble[0],np.mean(p_k_vaegan_ensemble,axis=0)
k_gan_ensemble,p_k_gan_ensemble = k_gan_ensemble[0],np.mean(p_k_gan_ensemble,axis=0)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10, 10), sharey=False)
plt.style.use('seaborn-ticks')
plt.rcParams["figure.figsize"] = (10,10)
sns.set_context("notebook")
# print(k)
# print(p_k)
plt.loglog(change_units(k_inputs),p_k_inputs,color='#e76f51')
plt.loglog(change_units(k_unet),p_k_unet,color='#e9c46a')
plt.loglog(change_units(k_vaegan_ensemble),p_k_vaegan_ensemble,color='#57cc99')
plt.loglog(change_units(k_gan_ensemble),p_k_gan_ensemble,color='#219ebc')
# plt.loglog(k_gan_ensemble[0],p_k_gan_ensemble[0],color='#219ebc')
plt.loglog(change_units(k_real),p_k_real,color='black')
axes.text(-0.1, 1.05, 'b.', transform=axes.transAxes, size=28, weight='bold')

# plt.xlim([10,2000])
# plt.xlabel("km ($1 / k$)",fontsize=30)
plt.xlabel("$k$ ($km^{-1}$)",fontsize=30)
plt.xlim([0,0.1])
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.ylabel("$P(k)$",fontsize=30)
plt.legend(['Bilinear','U-Net','VAEGAN','WGAN','HR obs'],frameon=True,fontsize=18)
# plt.fill_between(k_vaegan_ensemble, p_k_vaegan_min,p_k_vaegan_max,color='#57cc99',alpha=0.2)
# plt.fill_between(k_gan_ensemble, p_k_gan_min,p_k_gan_max,color='#219ebc',alpha=0.2)
plt.savefig('figure5-power_spectra-test-extreme_new_5.png',bbox_inches='tight',dpi=600)

In [ ]:
# print(change_units(k_inputs),p_k_inputs)
# print(change_units(k_unet),p_k_unet)
# print(change_units(k_vaegan_ensemble),p_k_vaegan_ensemble)
# print(change_units(k_gan_ensemble),p_k_gan_ensemble)
print(change_units(k_real),p_k_real)